Country Code:  
India - IN; Indonesia - ID; Mexico - MX; Colombia - CO  

In [4]:
!pip install pydeck -q -q

In [2]:
# Not working, since it is charged
import pydeck as pdk
import os 
os.environ['MAPBOX_API_KEY'] = "INSERT YOU MAPBOX TOKEN HERE"

In [9]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [5]:
pd.set_option("display.max_columns", None)

In [10]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}

# dl_table = f"{schema_name['cda']}.device_location"  
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

tj_table = f"{schema_name['cda']}.trajectory"     
pe_tj_table = f"{schema_name['cda']}.trajectory_uplevelled"

# stop_table = f"{schema['cda']}.stop" 
pe_stop_table = f"{schema_name['cda']}.stop_uplevelled"

visit_table = f"{schema_name['cda']}.visit " 


# Source table - Example

In [3]:
# Description
sql_engine.read_sql(f"desc {pe_dl_table}")

,Column,Type,Extra,Comment
0,event_timestamp,bigint,,
1,cuebiq_id,bigint,,
2,device_type_code,varchar(5),,
3,os_name,varchar,,
4,lat,double,,
5,lng,double,,
6,accuracy_meters,double,,
7,device_manufacturer_name,varchar,,
8,device_model_code,varchar,,
9,carrier_code,varchar,,


In [5]:
# Data
sql_engine.read_sql(
    f"""
    select * 
    from {pe_dl_table}
    where 
        country_code = 'CO'
        and processing_date = 20220201
        and provider_id = '700199'
    limit 5
    """
)

,event_timestamp,cuebiq_id,device_type_code,os_name,lat,lng,accuracy_meters,device_manufacturer_name,device_model_code,carrier_code,timezoneoffset_secs,admin1_id,admin2_id,speed_ms,bearing_angle_degree,floor_num,altitude_meters,event_zoned_datetime,classification_type,transformation_type,os_version,processing_date,country_code,provider_id
0,1643693574,5170255131,GAID,ANDROID,4.851021,-74.911599,110.00,Xiaomi,Redmi 7,None,-18000,CO.0011,CO.0011.0006,0.000000,0.00000,None,384.799988,2022-02-01T00:32:54-05:00,OTHER,KEEP,29,20220201,CO,700199
1,1643670647,5170255131,GAID,ANDROID,4.851023,-74.911586,20.00,Xiaomi,Redmi 7,None,-18000,CO.0011,CO.0011.0006,0.000000,0.00000,None,384.799988,2022-01-31T18:10:47-05:00,OTHER,KEEP,29,20220201,CO,700199
2,1643698144,5170255131,GAID,ANDROID,4.851024,-74.911587,180.00,Xiaomi,Redmi 7,None,-18000,CO.0011,CO.0011.0006,0.000000,0.00000,None,384.799988,2022-02-01T01:49:04-05:00,OTHER,KEEP,29,20220201,CO,700199
3,1643749763,5170254609,IDFA,IOS,0.000000,-74.494171,4.86,apple,"iPhone14,2",None,-18000,CO.0008,CO.0008.0002,17.830787,315.28266,None,159.667235,2022-02-01T16:09:23-05:00,OTHER,KEEP,15.2,20220201,CO,700199
4,1643758023,5170253507,IDFA,IOS,0.000000,-73.197968,4.58,apple,"iPhone11,8",None,-18000,CO.0008,CO.0008.0002,26.063644,243.52211,None,15.617821,2022-02-01T18:27:03-05:00,OTHER,KEEP,14.6,20220201,CO,700199


## Get provider_id

In [11]:
sql_engine.read_sql(
    f"""
    select 
        distinct provider_id
    from {pe_dl_table}
    where 
        country_code = 'ID'
        and processing_date BETWEEN 20190101 AND 20191231
    """
)

# MX & CO & ID & IN: 700199

,provider_id
0,700199


## Data

In [7]:
df = sql_engine.read_sql(
    f"""
    select * 
    from {pe_dl_table}
    where 
        country_code = 'CO'
        and processing_date = 20220201
        and provider_id = '700199'
    limit 5
    """
)
df

,event_timestamp,cuebiq_id,device_type_code,os_name,lat,lng,accuracy_meters,device_manufacturer_name,device_model_code,carrier_code,timezoneoffset_secs,admin1_id,admin2_id,speed_ms,bearing_angle_degree,floor_num,altitude_meters,event_zoned_datetime,classification_type,transformation_type,os_version,processing_date,country_code,provider_id
0,1643693574,5170255131,GAID,ANDROID,4.851021,-74.911599,110.00,Xiaomi,Redmi 7,None,-18000,CO.0011,CO.0011.0006,0.000000,0.00000,None,384.799988,2022-02-01T00:32:54-05:00,OTHER,KEEP,29,20220201,CO,700199
1,1643670647,5170255131,GAID,ANDROID,4.851023,-74.911586,20.00,Xiaomi,Redmi 7,None,-18000,CO.0011,CO.0011.0006,0.000000,0.00000,None,384.799988,2022-01-31T18:10:47-05:00,OTHER,KEEP,29,20220201,CO,700199
2,1643698144,5170255131,GAID,ANDROID,4.851024,-74.911587,180.00,Xiaomi,Redmi 7,None,-18000,CO.0011,CO.0011.0006,0.000000,0.00000,None,384.799988,2022-02-01T01:49:04-05:00,OTHER,KEEP,29,20220201,CO,700199
3,1643749763,5170254609,IDFA,IOS,0.000000,-74.494171,4.86,apple,"iPhone14,2",None,-18000,CO.0008,CO.0008.0002,17.830787,315.28266,None,159.667235,2022-02-01T16:09:23-05:00,OTHER,KEEP,15.2,20220201,CO,700199
4,1643758023,5170253507,IDFA,IOS,0.000000,-73.197968,4.58,apple,"iPhone11,8",None,-18000,CO.0008,CO.0008.0002,26.063644,243.52211,None,15.617821,2022-02-01T18:27:03-05:00,OTHER,KEEP,14.6,20220201,CO,700199


In [8]:
from datetime import datetime
import pytz
# df['event_zoned_datetime'][0]
datetime.fromisoformat(df['event_zoned_datetime'][0]).timestamp()

# event_timestamp is calculated from event_zoned_datetime

1643693574.0

## Column

In [9]:
sql_engine.read_sql(
    f"""
    select
        array_distinct(array_agg(classification_type)) classification_type,
        array_distinct(array_agg(transformation_type)) transformation_type
    from {pe_dl_table}
    where 
        country_code = 'MX'
        and processing_date = 20190101
        and provider_id = '700199'
        and cuebiq_id % 1000 = 0
    group by 
        country_code
    """
)

# this one I have't understand yet.

,classification_type,transformation_type
0,"[OTHER, RECURRING_AREA, WHITELISTED]","[KEEP, UPLEVELED]"


In [19]:
# Get certiain columns 
df_points = sql_engine.read_sql(
    f"""
    select 
        lat, 
        lng
    from {pe_dl_table}
    where 
        country_code = 'CO'
        and processing_date = 20220201
        and provider_id = '700199'
        
        and cuebiq_id % 1000 = 0
    """
)
df_points

,lat,lng
0,0.000000,-75.299469
1,0.000000,-75.131500
2,0.000000,-75.079376
3,0.000000,-74.128998
4,0.000000,-74.199615
5,0.000000,-74.048431
6,4.600705,-74.125211
7,4.607533,-74.133242
8,4.608474,-74.133539
9,4.608782,-74.135358


# Read in Data

## Overall yearly  

### single

In [28]:
pe_dl_df = sql_engine.read_sql(
    f"""
    select *
    from {pe_dl_table}
    where 
        country_code = 'CO'
        and processing_date = 20190101

    """
)
# and cuebiq_id % 1000 = 0

pe_dl_df

,event_timestamp,cuebiq_id,device_type_code,os_name,lat,lng,accuracy_meters,device_manufacturer_name,device_model_code,carrier_code,...,bearing_angle_degree,floor_num,altitude_meters,event_zoned_datetime,classification_type,transformation_type,os_version,processing_date,country_code,provider_id
0,1546348958,1697829916,IDFA,IOS,0.0,-75.272186,5.0,,,,...,None,None,None,2019-01-01T08:22:38-05:00,OTHER,KEEP,12.1,20190101,CO,700199
1,1546325692,1697822319,IDFA,IOS,0.0,-73.496735,6.0,,,,...,None,None,None,2019-01-01T01:54:52-05:00,OTHER,KEEP,12.1.2,20190101,CO,700199
2,1546325692,1697822319,IDFA,IOS,0.0,-73.496735,6.0,,,,...,None,None,None,2019-01-01T01:54:52-05:00,OTHER,KEEP,12.1.2,20190101,CO,700199
3,1546353747,1697595594,IDFA,IOS,0.0,-75.152069,65.0,,,,...,None,None,None,2019-01-01T09:42:27-05:00,OTHER,KEEP,12.1.2,20190101,CO,700199
4,1546353747,1697595594,IDFA,IOS,0.0,-75.152069,65.0,,,,...,None,None,None,2019-01-01T09:42:27-05:00,OTHER,KEEP,12.1.2,20190101,CO,700199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43320,1546372595,1124769826,IDFA,IOS,0.0,-73.583527,65.0,,,,...,None,None,None,2019-01-01T14:56:35-05:00,OTHER,KEEP,12.1.2,20190101,CO,700199
43321,1546368132,1124769826,IDFA,IOS,0.0,-73.582367,5.0,,,,...,None,None,None,2019-01-01T13:42:12-05:00,OTHER,KEEP,12.1.2,20190101,CO,700199
43322,1546301711,1124769826,IDFA,IOS,0.0,-73.577179,65.0,,,,...,None,None,None,2018-12-31T19:15:11-05:00,OTHER,KEEP,12.1.2,20190101,CO,700199
43323,1546320705,1125288098,IDFA,IOS,0.0,-74.846405,5.0,,,,...,None,None,None,2019-01-01T00:31:45-05:00,OTHER,KEEP,12.1,20190101,CO,700199


### tj_table

In [10]:
sql_engine.read_sql(f"desc {tj_table}")

,Column,Type,Extra,Comment
0,cuebiq_id,bigint,,
1,device_type_code,varchar(5),,
2,os_name,varchar,,
3,start_zoned_datetime,varchar,,
4,start_geohash,varchar,,
5,start_lat,double,,
6,start_lng,double,,
7,start_country,varchar,,
8,start_admin1,varchar,,
9,start_admin2,varchar,,


In [10]:
country_code = 'IN'

# Start the timer
start_time = time.time()

result = sql_engine.read_sql(
    f"""
    SELECT 
        count(*) AS Total_Observations,
        count(DISTINCT cuebiq_id) AS Total_Unique_Users
    
    FROM {tj_table}
    WHERE 
        start_country = '{country_code}'
        AND end_country = '{country_code}'
        AND event_date BETWEEN 20190101 AND 20191231
    """)

# Stop the timer
end_time = time.time()

# Calculate the total time taken
execution_time = end_time - start_time

# Extract the total observations and unique users from the result
total_observations = result.iloc[0]['Total_Observations']
total_unique_users = result.iloc[0]['Total_Unique_Users']

# Print the results
print(f"Total Observations: {total_observations}")
print(f"Total Unique Users: {total_unique_users}")

# Print the execution time
print(f"Query executed in: {execution_time:.2f} seconds")

Total Observations: 98511275
Total Unique Users: 2382603
Query executed in: 1466.47 seconds


### pe_dl_table

In [11]:
sql_engine.read_sql(f"desc {pe_dl_table}")

,Column,Type,Extra,Comment
0,event_timestamp,bigint,,
1,cuebiq_id,bigint,,
2,device_type_code,varchar(5),,
3,os_name,varchar,,
4,lat,double,,
5,lng,double,,
6,accuracy_meters,double,,
7,device_manufacturer_name,varchar,,
8,device_model_code,varchar,,
9,carrier_code,varchar,,


In [17]:
country_code = 'CO'

In [18]:
result = sql_engine.read_sql(
    f"""
    SELECT 
        count(*) AS Total_Observations,
        count(DISTINCT cuebiq_id) AS Total_Unique_Users
    FROM (
        SELECT 
            try(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s') +
                interval '1' second * timezoneoffset_secs) AS event_datetime_utc,
            cuebiq_id
        FROM {pe_dl_table}
        WHERE 
            country_code = '{country_code}'
            AND event_zoned_datetime IS NOT NULL
            AND processing_date BETWEEN 20181201 AND 20200131
            AND length(event_zoned_datetime) >= 19
    ) AS subquery
    WHERE 
        event_datetime_utc IS NOT NULL
        AND event_datetime_utc BETWEEN date '2019-01-01' AND date '2019-12-31'
    """
)

print(result)


   Total_Observations  Total_Unique_Users
0           152150003              521012


### pe_stop_table

In [7]:
sql_engine.read_sql(f"desc {pe_stop_table}")

,Column,Type,Extra,Comment
0,cuebiq_id,bigint,,
1,device_type_code,varchar(5),,
2,os_name,varchar,,
3,lat,double,,
4,lng,double,,
5,geohash_id,varchar,,
6,cluster_size,integer,,
7,avg_distance_meters,double,,
8,avg_accuracy_meters,double,,
9,std_accuracy_meters,double,,


In [22]:
pe_dl_df = sql_engine.read_sql(
    f"""
    select *
    from {pe_stop_table}
    where 
        country_code = 'MX'
        and processing_date = 20190101

    """
)

pe_dl_df

,cuebiq_id,device_type_code,os_name,lat,lng,geohash_id,cluster_size,avg_distance_meters,avg_accuracy_meters,std_accuracy_meters,...,stop_zoned_datetime,timezone_offset_seconds,admin1_id,admin2_id,classification_type,transformation_type,block_group_id,processing_date,country_code,provider_id
0,1698772522,GAID,ANDROID,19.259637,-99.582992,9g3jtfjgs,8,1.721615e+01,55.125000,16.357065,...,2018-12-31T18:04:58-06:00,-21600,MX.0020,MX.0020.0003,OTHER,KEEP,None,20190101,MX,700199
1,1698772522,GAID,ANDROID,19.261112,-99.582634,9g3jtfqhh,2,1.983196e+01,51.000000,29.698485,...,2018-12-31T20:33:09-06:00,-21600,MX.0020,MX.0020.0003,OTHER,KEEP,None,20190101,MX,700199
2,1698772522,GAID,ANDROID,19.261170,-99.582565,9g3jtfqhq,6,1.125673e+01,28.666667,2.065591,...,2018-12-31T22:20:53-06:00,-21600,MX.0020,MX.0020.0003,OTHER,KEEP,None,20190101,MX,700199
3,1698772522,GAID,ANDROID,19.267325,-99.552767,9g3jweebk,4,1.169287e+01,36.500000,10.472185,...,2018-12-31T21:02:08-06:00,-21600,MX.0020,MX.0020.0006,OTHER,KEEP,None,20190101,MX,700199
4,1698771601,GAID,ANDROID,18.611925,-99.256473,9g33vk5wk,7,2.866105e+01,92.571429,21.923026,...,2018-12-31T20:28:29-06:00,-21600,MX.0023,MX.0023.0011,OTHER,KEEP,None,20190101,MX,700199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680486,1374931477,GAID,ANDROID,21.361401,-102.850325,9eyce8dxj,7,1.686926e+01,36.285714,16.509737,...,2018-12-31T00:24:01-06:00,-21600,MX.0007,MX.0007.0001,OTHER,KEEP,None,20190101,MX,700199
680487,1374931477,GAID,ANDROID,21.361533,-102.850359,9eyce8dxu,3,1.716642e+01,74.666667,46.188022,...,2018-12-31T15:33:18-06:00,-21600,MX.0007,MX.0007.0001,OTHER,KEEP,None,20190101,MX,700199
680488,1374931477,GAID,ANDROID,21.360425,-102.849065,9eyce8e97,1,1.473278e-09,24.000000,0.000000,...,2018-12-31T14:58:19-06:00,-21600,MX.0007,MX.0007.0001,OTHER,KEEP,None,20190101,MX,700199
680489,1374931477,GAID,ANDROID,21.364979,-102.844416,9eyce9qup,1,1.473232e-09,96.000000,0.000000,...,2018-12-31T14:43:25-06:00,-21600,MX.0007,MX.0007.0001,OTHER,KEEP,None,20190101,MX,700199


In [27]:
country_code = 'MX'

start_time = time.time() # Start the timer

result = sql_engine.read_sql(
    f"""
    SELECT 
        count(*) AS Total_Observations,
        count(DISTINCT cuebiq_id) AS Total_Unique_Users
    FROM (
        SELECT 
            try(date_parse(substr(stop_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s') +
                interval '1' second * timezone_offset_seconds) AS event_datetime_utc,
            cuebiq_id
        FROM {pe_stop_table}
        WHERE 
            country_code = '{country_code}'
            AND stop_zoned_datetime IS NOT NULL
            AND processing_date BETWEEN 20181201 AND 20200131
            AND length(stop_zoned_datetime) >= 19
    ) AS subquery
    WHERE 
        event_datetime_utc IS NOT NULL
        AND event_datetime_utc BETWEEN date '2019-01-01' AND date '2019-12-31'
    """
)

end_time = time.time() # Stop the timer
execution_time = end_time - start_time # Calculate the total time taken
print(f"Query executed in: {execution_time:.2f} seconds")

result


Query executed in: 45.28 seconds


,Total_Observations,Total_Unique_Users
0,292956571,3437467


In [11]:
###########################  删  ######################

country_code = 'IN'

# Start the timer
start_time = time.time()

result = sql_engine.read_sql(
    f"""
    SELECT 
        count(*) AS Total_Observations,
        count(DISTINCT cuebiq_id) AS Total_Unique_Users
    
    FROM {pe_stop_table}
    WHERE 
        country_code = '{country_code}'
        AND processing_date BETWEEN 20190101 AND 20191231
    """)

# Stop the timer
end_time = time.time()

# Calculate the total time taken
execution_time = end_time - start_time

# Extract the total observations and unique users from the result
total_observations = result.iloc[0]['Total_Observations']
total_unique_users = result.iloc[0]['Total_Unique_Users']

# Print the results
print(f"Total Observations: {total_observations}")
print(f"Total Unique Users: {total_unique_users}")

# Print the execution time
print(f"Query executed in: {execution_time:.2f} seconds")

Total Observations: 220177858
Total Unique Users: 3159043
Query executed in: 16.15 seconds


### visit_table

In [12]:
sql_engine.read_sql(f"desc {visit_table}")

,Column,Type,Extra,Comment
0,admin1_id,varchar,,
1,admin2_id,varchar,,
2,brand_id,bigint,,
3,country_code,varchar,,
4,cuebiq_id,bigint,,
5,dwell_time_minutes,double,,
6,event_date,integer,,
7,geohash,varchar,,
8,geoset_id,integer,,
9,lat,double,,


In [30]:
df = sql_engine.read_sql(
    f"""
    select * 
    from {visit_table}
    where 
        country_code = 'IN'
    """
)
df.to_csv(f'/home/jovyan/Data/IN_visit.csv', index=False)  

In [20]:
country_code = 'CO'

start_time = time.time() # Start the timer

result = sql_engine.read_sql(
    f"""
    SELECT 
        count(*) AS Total_Observations,
        count(DISTINCT cuebiq_id) AS Total_Unique_Users
    
    FROM {visit_table}
    WHERE 
        country_code = '{country_code}'
        AND event_date BETWEEN 20230101 AND 20231231
    """)

end_time = time.time()  # Stop the timer
execution_time = end_time - start_time  # Calculate the total time taken

# Extract the total observations and unique users from the result
total_observations = result.iloc[0]['Total_Observations']
total_unique_users = result.iloc[0]['Total_Unique_Users']

# Print the results
print(f"Total Observations: {total_observations}")
print(f"Total Unique Users: {total_unique_users}")

# Print the execution time
print(f"Query executed in: {execution_time:.2f} seconds")

Total Observations: 0
Total Unique Users: 0
Query executed in: 312.07 seconds
